# Assignment 2

## Task 1

In [1]:

from ucimlrepo import fetch_ucirepo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)


ConnectionError: Error connecting to server

## Task 2

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
X.shape

### Task 2.1

In [ ]:
X.hist(figsize=(16, 16))
plt.show()

## Task 3

In [ ]:
X.isna().sum()

## Task 4

In [ ]:
X = X.replace('?', np.nan)
X.info()

## Task 5

In [ ]:
# Selecting a list of number columns and the categorical columns separately
num_cols = X.select_dtypes(include='number').columns.to_list()
cat_cols = X.select_dtypes(exclude='number').columns.to_list()


##Making a pipeline with replaces the data using the correct strategy
num_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(sparse_output=False))

#creating the column transformer
preprocessing = ColumnTransformer([('num', num_pipeline, num_cols), ('cat', cat_pipeline, cat_cols)])

preprocessing


In [ ]:
#applying the transformer to the values
X_prepared = preprocessing.fit_transform(X)
#getting the columns which were stripped
feature_names = preprocessing.get_feature_names_out()
#adding them back
X_prepared = pd.DataFrame(data=X_prepared, columns=feature_names)
X_prepared

In [ ]:
X_prepared.shape


## Task 6

In [ ]:
y.value_counts()


## Task 7

In [ ]:
y_prepared = y.replace('>50K.', '>50K')
y_prepared = y_prepared.replace('<=50K.', '<=50K')
y_prepared.value_counts()


## Task 8

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_prepared, y_prepared, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Task 9

In [ ]:
from sklearn.svm import SVC
model_svm = SVC(kernel='poly', C=0.1, gamma=1)
#Only first 1000 as it's slow
model_svm.fit(X_train.iloc[:1000], y_train.iloc[:1000].values.ravel())

### Task 9.1

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
y_predict = model_svm.predict(X_test)
print(classification_report(y_test, y_predict))

### Task 9.2

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_predict)

## Task 10
### Task 10.1

In [ ]:
## Split data into 60% train
X_train, X_validation_test, y_train, y_validation_test = train_test_split(X_prepared, y_prepared, test_size=0.4, random_state=42)
## Split remaining into 50% test 50% validation (overall 20% each)
X_validation, X_test, y_validation, y_test = train_test_split(X_validation_test, y_validation_test, test_size=0.5, random_state=42)
print(X_train.shape, y_train.shape, X_validation.shape, y_validation.shape, X_test.shape, y_test.shape)

### Task 10.2

In [ ]:
from sklearn.model_selection import GridSearchCV

svm_parameters = {'kernel':['rbf'],
                  'C': [0.01, 0.1, 1, 10],
                  'gamma': [0.01, 1, 10]}
svm= SVC()
svm_gs =GridSearchCV(estimator=svm, param_grid=svm_parameters)
svm_gs.fit(X_train.iloc[:1000], y_train.iloc[:1000].values.ravel())

svm_winner = svm_gs.best_estimator_
svm_winner.score(X_validation, y_validation)

## Task 10.3

In [ ]:
svm_winner